<a href="https://colab.research.google.com/github/leeds1219/Article_Review/blob/main/AudioVisual_Grouping_Network.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **Audio-Visual Grouping Network for Sound Localization from Mixtures**
Shentong MO, Yapeng Tian


<img src = "https://drive.google.com/uc?id=1tPSL_zpnv99tELSvOgUkbmE69Xr6uvNF">


C is the source event categories

Aggregate audio-visual tokens $\{ \hat{c}_{i}^{a} \}_{i=1}^{C}$,
Global audio and spatial visual features $\hat{f}^{a}$


Align features and tokens from raw input of image and audio mixture with self attention transformers: $\hat{f}^{a}$, $\{ \hat{c}_{i}^{a} \}_{i=1}^{C}$ = $\{\phi(x_{j}^{a}, X^{a},X^{a})\}_{j=1}^{1+C}$

$\text{Attention}(Q, K, V) = \text{Softmax}\left(\frac{QK^T}{\sqrt{d_k}}\right) V
$

$X^{a}$ = $\{x_{j}^{a}\}_{j=1}^{1+C}$ = [$f^{a}$;$\{c_{i}\}_{i=1}^{C}$] is the entire set of input features, $x_{j}^{a}$ is the indiviual element in the set.

${\phi^{a}}(x_{j}^{a}, X^{a},X^{a}) = \text{Softmax}\left(\frac{{x_{j}^{a}}{X^{a}}^T}{\sqrt{D}}\right) {X^{a}}
$

FC layer and add softmax operator